In [1]:
import numpy as np

In [2]:
import numpy as np
import random

class SimpleGPT:
    def __init__(self, vocab_size: int, embed_dim: int, seq_length: int):
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.seq_length = seq_length
        
        self.token_embed = np.random.randn(vocab_size, embed_dim) * 0.1
        self.pos_embed = np.random.randn(seq_length, embed_dim) * 0.1
        
        self.Wq = np.random.randn(embed_dim, embed_dim) * 0.1
        self.Wk = np.random.randn(embed_dim, embed_dim) * 0.1
        self.Wv = np.random.randn(embed_dim, embed_dim) * 0.1
        
        self.W_out = np.random.randn(embed_dim, vocab_size) * 0.1
        
    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return exp_x / exp_x.sum(axis=-1, keepdims=True)
    
    def attention(self, x):
        Q = np.dot(x, self.Wq)
        K = np.dot(x, self.Wk)
        V = np.dot(x, self.Wv)
        
        scores = np.dot(Q, K.T) / np.sqrt(self.embed_dim)
        attn = self.softmax(scores)
        return np.dot(attn, V)
    
    def forward(self, inputs):
        token_emb = self.token_embed[inputs]
        pos_emb = self.pos_embed[:len(inputs)]
        x = token_emb + pos_emb
        x = self.attention(x)
        logits = np.dot(x, self.W_out)
        return self.softmax(logits[-1])
    
    def backward(self, inputs, x, Q, K, V, attn, attn_output, probs, target, lr):
        # Градиенты логитов
        d_logits = probs.copy()
        d_logits[target] -= 1

        d_W_out = np.outer(attn_output[-1], d_logits)
        d_attn_out = np.zeros_like(attn_output)
        d_attn_out[-1] = np.dot(self.W_out, d_logits)

        d_V = np.dot(attn.T, d_attn_out)
        d_attn = np.dot(d_attn_out, V.T)

        d_scores = attn * (d_attn - (attn * d_attn).sum(axis=1, keepdims=True))
        d_scores /= np.sqrt(self.embed_dim)

        d_Q = np.dot(d_scores, K)
        d_K = np.dot(d_scores.T, Q)

        d_Wq = np.dot(x.T, d_Q)
        d_Wk = np.dot(x.T, d_K)
        d_Wv = np.dot(x.T, d_V)

        d_emb = np.dot(d_Q, self.Wq.T) + np.dot(d_K, self.Wk.T) + np.dot(d_V, self.Wv.T)

        self.W_out -= lr * d_W_out
        self.Wq -= lr * d_Wq
        self.Wk -= lr * d_Wk
        self.Wv -= lr * d_Wv
        self.token_embed[inputs] -= lr * d_emb
        self.pos_embed[:len(inputs)] -= lr * d_emb
    def train_step(self, inputs, target, lr):
        token_emb = self.token_embed[inputs]
        pos_emb = self.pos_embed[:len(inputs)]
        x = token_emb + pos_emb

        Q = np.dot(x, self.Wq)
        K = np.dot(x, self.Wk)
        V = np.dot(x, self.Wv)

        scores = np.dot(Q, K.T) / np.sqrt(self.embed_dim)
        attn = self.softmax(scores)
        attn_output = np.dot(attn, V)

        logits = np.dot(attn_output, self.W_out)
        probs = self.softmax(logits[-1])

        loss = -np.log(probs[target])

        self.backward(inputs, x, Q, K, V, attn, attn_output, probs, target, lr)

        return loss
    def generate(self, start_seq, char_to_idx, idx_to_char, max_new_tokens):
        generated = start_seq[:]
        for _ in range(max_new_tokens):
            context = generated[-self.seq_length:]
            if len(context) < self.seq_length:
                context = [0] * (self.seq_length - len(context)) + context
            probs = self.forward(context)
            next_token = np.argmax(probs)
            generated.append(next_token)
        return ''.join([idx_to_char[i] for i in generated])

In [3]:
def text_to_indices(text, char_to_idx):
    return [char_to_idx[c] for c in text]

In [4]:
text = """«Текст» — российский криминально-драматический психологический триллер режиссёра Клима Шипенко,
экранизация романа-бестселлера «Текст» (2017) писателя Дмитрия Глуховского, который сам адаптировал свой роман в киносценарий.
Фильм рассказывает о бывшем заключённом Илье Горюнове, который мстит полицейскому Петру Хазину, подбросившему ему наркотики,
и в результате получает доступ к его смартфону. Вместе с ним он получает доступ и к жизни героя, и на время становится для всех Хазиным,
отправляя сообщения его начальству, родителям и девушке Нине, в которую влюбляется и сам."""*500
chars = sorted(list(set(text)))
vocab_size = len(chars)
char_to_idx = {c:i for i,c in enumerate(chars)}
idx_to_char = {i:c for c,i in char_to_idx.items()}
seq_length = 5

X, y = [], []
for i in range(len(text) - seq_length):
    X.append(text_to_indices(text[i:i+seq_length], char_to_idx))
    y.append(char_to_idx[text[i+seq_length]])

model = SimpleGPT(vocab_size, embed_dim=16, seq_length=seq_length)

epochs = 100
learning_rate = 0.0005
for epoch in range(epochs):
    total_loss = 0
    for inputs, target in zip(X, y):
        loss = model.train_step(inputs, target, learning_rate)
        total_loss += loss
    if epoch % 10 == 0 or epoch == epochs - 1:
        print(f"Epoch {epoch}, Loss: {total_loss/len(X)}")

Epoch 0, Loss: 3.4177491331244263
Epoch 10, Loss: 0.8090330833228876
Epoch 20, Loss: 0.5436244989883153
Epoch 30, Loss: 0.46484287366960636
Epoch 40, Loss: 0.4575337757083219
Epoch 50, Loss: 0.42037114036152246
Epoch 60, Loss: 0.37320764808148277
Epoch 70, Loss: 0.38295748188304407
Epoch 80, Loss: 0.34607843141582834
Epoch 90, Loss: 0.3328544664190199
Epoch 99, Loss: 0.3256029507149844


In [5]:
start_text = "Текст"
test_input = [char_to_idx[c] for c in start_text]
generated_text = model.generate(test_input, char_to_idx, idx_to_char, max_new_tokens=100)
print(f"Generated text:\n{generated_text}")

Generated text:
Текст» — российский триоваляя в киносценартфону. Вместе с н вуПеру наркотики,
ровившему ему наркотики,
ро


In [6]:
start_text = "Петру"
test_input = [char_to_idx[c] for c in start_text]
generated_text = model.generate(test_input, char_to_idx, idx_to_char, max_new_tokens=100)
print(f"Generated text:\n{generated_text}")

Generated text:
Петру Хазиным,
отправучс.ческо агояоммес адаптироваляя в киносценартфону. Вместе с н вуПеру наркотики,
ро


In [7]:
start_text = "Клима"
test_input = [char_to_idx[c] for c in start_text]
generated_text = model.generate(test_input, char_to_idx, idx_to_char, max_new_tokens=100)
print(f"Generated text:\n{generated_text}")

Generated text:
Клима Шипенко о,
бщнатсИльтьте ну Нине, в который.омус агояоммес адаптироваляя в киносценартфону. Вместе 
